In [4]:
import numpy as np

In [9]:
from src.generate_mesh import triangular_lattice
from src.generate_mesh import create_mesh
from src.elastic_energy import elastic_energy
from src.MC_vertex_link import MCstep_vertex
from src.MC_vertex_link import MCstep_link

In [6]:
triangular_lattice = np.array( triangular_lattice(1.1, expansion_level = 5) ) # returns a list with points on the Euclidean plane (2d space)
ver, TRI=create_mesh(triangular_lattice)

1
(-1.6500000000000001, 0.9526279441628825)
1
(-2.75, 0.9526279441628824)
2
(-2.2, 1.9052558883257649)
1
(-3.8500000000000005, 0.9526279441628825)
2
(-3.3000000000000003, 1.905255888325765)
3
(-2.75, 2.8578838324886475)
1
(-4.95, 0.9526279441628825)
2
(-4.4, 1.905255888325765)
3
(-3.8499999999999996, 2.857883832488648)
4
(-3.3, 3.81051177665153)


In [11]:
k=0.000002
β=0.1
r=1.0
σ=0.1*r
for z in range(0,1000): #Some thermalization
#     MCstep_vertex(ver,TRI,σ,r,k,β)
    MCstep_link(ver,TRI,β,k,r)

KeyboardInterrupt: 